### Projects of using Spark
#### 16300200020 Yanjian Zhang

Use the linux command to delete the first 76 lines and redirect to the data_new.txt file.Then delete the last 49 lines and redirect to the data_new.txt. We can see that the total numbers of lines are 49611709

Introduce the spark package and configure the spark context.

In [1]:
import os
os.getcwd()

'/home/bigdatalab06'

In [2]:
from __future__ import print_function,division
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from itertools import islice
import re
conf = SparkConf().setAppName("turkey").setMaster("local").set("spark.executor.memory","12g")
conf.set("spark.executor.cores","4").set("spark.memory.offHeap.enabled","true")
conf.set("spark.executor.memoryOverhead","4096").set("spark.memory.offHeap.size","4096")
sc = SparkContext(conf=conf)


In [18]:
from pyspark.ml.linalg import Vector,Vectors
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer,VectorIndexer
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import pyspark.sql as sql

data = sc.textFile('hdfs://localhost:9001/user/data_4000w.txt').map(lambda x : re.split("\t", x))
# classOfFirstName = data.map(lambda x: x[2]).distinct().persist()
# print(classOfFirstName.take(100))

In [19]:
# print(len(classOfFirstName))

In [20]:
# classOfLastName = data.map(lambda x: x[3]).distinct().persist()

In [21]:
from pyspark.sql import Row
spark = SparkSession(sc)
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer


In [22]:
def label2onehot(label,len_):
    onehot = [0] * len_
    onehot[int(label)] = 1
    return onehot
from pyspark.ml.feature import OneHotEncoderEstimator
currentdata = data.map(lambda x: [x[6], x[2]])
print(currentdata.take(10))
# highFre = data.map(lambda x: (x[2], 1)).reduceByKey(lambda x, y: x+y).top(200000, key = lambda x: x[1])
# print(data.map(lambda x: x[14]).distinct().count()) # 207482
# print(data.map(lambda x: x[14].split()[0]).distinct().count()) # 119346
schema = ['gender','first_name']
# forfit = spark.createDataFrame(currentdata.filter(lambda x: x[1] in highFre), schema)
df = spark.createDataFrame(currentdata, schema)

indexer = [StringIndexer(inputCol = column, outputCol = column + "_index") for column in schema]
# onehotindexer = OneHotEncoderEstimator(inputCols = [column + "_index" for column in schema], outputCols = [column + "_onehot" for column in schema])
pipeline = Pipeline(stages = indexer)
df = pipeline.fit(df).transform(df)
df = df.drop(*schema)
df.show()


[['K', 'NESLIHAN'], ['K', 'SADET'], ['K', 'GONUL'], ['E', 'MURAT'], ['K', 'SENEM'], ['K', 'ZOHRE'], ['K', 'ARIFE'], ['K', 'AYSE'], ['K', 'MELEK'], ['K', 'YILDIZ']]
+------------+----------------+
|gender_index|first_name_index|
+------------+----------------+
|         0.0|           170.0|
|         0.0|           748.0|
|         0.0|            97.0|
|         1.0|            11.0|
|         0.0|           397.0|
|         0.0|           510.0|
|         0.0|           190.0|
|         0.0|             2.0|
|         0.0|            55.0|
|         0.0|           112.0|
|         1.0|           375.0|
|         1.0|            39.0|
|         1.0|            53.0|
|         1.0|           107.0|
|         1.0|             0.0|
|         1.0|            80.0|
|         0.0|            40.0|
|         1.0|           498.0|
|         0.0|          2851.0|
|         1.0|           323.0|
+------------+----------------+
only showing top 20 rows



In [8]:
def hit_n(l_, n):
    count = 0
    record = {}
    for item in l_:
        record[count] = item
        count += 1
    record  = sorted(record.items() , key = lambda x:x[1], reverse = True)
    result = []
    for i in range(n):
        result.append(record[i][0])
    return result
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes

vectorForm = df.rdd.map(lambda row: (row[0], Vectors.dense(row[1:2])))
Vectordf = spark.createDataFrame(vectorForm, ['label', 'features'])
trainData, validData, testData = Vectordf.rdd.randomSplit([0.7,0.1,0.2])
# linear regression
# lr = LogisticRegression(regParam=0.1, elasticNetParam=1.0, family="multinomial")
# lrmodel = lr.fit(trainData.toDF())
# randomforest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=2)
rfmodel = rf.fit(trainData.toDF())
predictions = rfmodel.transform(testData.toDF())
# bayes
# nb = NaiveBayes(featuresCol="features", labelCol="label")
# nbmodel = nb.fit(trainData.toDF())
# predictions = nbmodel.transform(testData.toDF())
p1 = predictions.select('probability', 'label')

In [9]:
p1.show(30)

+--------------------+-----+
|         probability|label|
+--------------------+-----+
|[0.53775505166535...|  0.0|
|[0.66461526699637...|  0.0|
|[0.66461526699637...|  1.0|
|[0.53775505166535...|  1.0|
|[0.53775505166535...|  1.0|
|[0.53775505166535...|  1.0|
|[0.53775505166535...|  0.0|
|[0.65933147469178...|  0.0|
|[0.53775505166535...|  1.0|
|[0.53775505166535...|  1.0|
|[0.64171891715466...|  0.0|
|[0.52413583516642...|  1.0|
|[0.53775505166535...|  1.0|
|[0.66461526699637...|  1.0|
|[0.14706768174629...|  1.0|
|[0.52413583516642...|  1.0|
|[0.14706768174629...|  0.0|
|[0.14706768174629...|  1.0|
|[0.39298099239038...|  0.0|
|[0.52413583516642...|  1.0|
|[0.14706768174629...|  1.0|
|[0.14706768174629...|  1.0|
|[0.14706768174629...|  1.0|
|[0.14706768174629...|  0.0|
|[0.65933147469178...|  0.0|
|[0.64171891715466...|  1.0|
|[0.52413583516642...|  1.0|
|[0.14706768174629...|  1.0|
|[0.14706768174629...|  1.0|
|[0.14706768174629...|  1.0|
+--------------------+-----+
only showing t

In [10]:
# print(p1.rdd.map(lambda x: (list(enumerate(x[0])), x[1])).take(10))
hits = p1.rdd.map(lambda x: (sorted(enumerate(x[0]), key = lambda y: -y[1]), x[1])).map(lambda row: ([x[0] for x in row[0]], row[1])) # 按index 排序
hits.take(10)
hits = hits.map(lambda x: [ 1 if x[0][i]== x[1] else 0 for i in range(2)])# 计算1-5的hit
total = hits.count()
hits = [hits.filter(lambda x: x[i] == 1).count()/total for i in range(2)]
print(hits)
# hits.take(2000)

[0.6220115951730908, 0.3779884048269092]


In [11]:
# topn = [p1.rdd.map(lambda x: (hit_n(x[0],1),x[1])).filter(lambda x: int(x[1]) in x[0]).count()/float(top1_pre.count()) for i]

In [12]:
# print(top1_pre.take(10))
# print(top1_pre.filter(lambda x: int(x[1]) in x[0]).count()/float(top1_pre.count()))